In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Concatenate, Dense, Activation, TextVectorization, Embedding, GlobalAveragePooling1D, Conv1D, AveragePooling1D, Reshape
from keras.models import Model
import tensorflow as tf

In [17]:
df = pd.read_csv('datasets/IMDB Dataset.csv')#.sample(1000)
ohe = OneHotEncoder()
y_ohe = ohe.fit_transform(df['sentiment'].to_numpy().reshape((-1,1))).todense()
X_train, X_test, y_train, y_test = train_test_split(df['review'], y_ohe)

# Redes Convolucionais

Uma convolução é um tipo de operação matemática que funciona aplicando um filtro $h$ sobre um sinal $x$.

O sinal $x$ tem $T$ amostras ao longo do tempo e tem $D$ dimensões, portanto é armazenado numa matriz $x \in \mathbb{R}^{T \times D}$.

O filtro $h$ tem $K$ amostras ao longo do tempo e tem as mesmas $D$ dimensões.

A ideia da convolução é o filtro é posicionado sobre o sinal à partir do tempo inicial, seus valores são multiplicados ponto-a-ponto e então somados. Isso gera a saída da convolução para o tempo 0. Depois disso, o filtro é deslocado em uma amostra de tempo e a operação é repetida, gerando o resultado para o tempo 1. Em outras palavras:

$$
y[n] = \sum_{d=0}^{D-1} \sum_{k=0}^{K-1} x[n+k, d] h[k, d]
$$

As convoluções realizadas em espaços de embeddings de palavras podem ser semelhantes a usar n-gramas, pois um filtro $h[k,d]$ descreve uma trajetória no espaço de embeddings.

Uma outra propriedade das convoluções é que, como elas "agrupam" várias amostras de $x$ em cada amostra de $y$, é possível sub-amostrar $y$ após seu cálculo sem perder informações.

Por fim, como a convolução é equivalente à aplicação de uma rede neural, podemos usar não-linearidades na saída para aumentar a capacidade expressiva da rede.

## Exercício 1
**Objetivo: desenhar o processo de convolução e sub-amostragem**

No código abaixo, descrevemos uma pequena camada de rede que usa 3 outras camadas e realiza uma convolução, sub-amostra o sinal convoluído e, por fim, aplica uma função de ativação.

1. Identifique as três etapas relacionadas a essa camada.

1. Justifique as dimensões das entradas e saídas desta camada.

In [82]:
def convolve_and_downsample(input_n_samples, input_embedding_size, n_filters, kernel_size=3, **kwargs):
    input_layer = Input(shape=(input_n_samples,input_embedding_size))
    x = input_layer
    x = Conv1D( filters=n_filters,
                kernel_size=kernel_size,
                padding='same',
                use_bias=False,
                )(x)
    x = AveragePooling1D(pool_size=2)(x)
    x = Activation('elu')(x)
    return Model(input_layer, x, **kwargs)

cds = convolve_and_downsample(8, 2, 4, 3, name='ngrama')
print(cds.summary())

Model: "ngrama"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_93 (InputLayer)       [(None, 8, 2)]            0         
                                                                 
 conv1d_74 (Conv1D)          (None, 8, 4)              24        
                                                                 
 average_pooling1d_74 (Avera  (None, 4, 4)             0         
 gePooling1D)                                                    
                                                                 
 activation_62 (Activation)  (None, 4, 4)              0         
                                                                 
Total params: 24
Trainable params: 24
Non-trainable params: 0
_________________________________________________________________
None


## Exercício 2
**Objetivo: usar uma camada customizada numa rede neural**

No código abaixo, verifique:

1. Como é possível usar a camada de rede customizada que fizemos no ex.1 dentro de uma rede neural mais ampla?
1. Quantos filtros diferentes são usados para calcular n-gramas?
1. Já vimos que o número de n-gramas possíveis é proibitivamente alto para qualquer $n$ maior que 2 ou 3. Como os filtros se diferenciam dos n-gramas para evitar essa explosão?
1. Como as curvas de treinamento do modelo convolucional se diferenciam dos que vimos anteriormente, usando somente média ou usando redes recorrentes?

In [90]:
vocab_size = 1000
def cnn_embedding_softmax_model(vectorize_layer, vocab_size=vocab_size):
    input_layer = Input(shape=(1,), dtype=tf.string)
    x = input_layer
    x = vectorize_layer(x)
    x = Embedding(vocab_size, 2, name='projecao')(x)
    x = convolve_and_downsample(256, 2, 16, 3, name='ngramas')(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(2, name='classificador')(x)
    x = Activation('softmax')(x)
    return Model(input_layer, x)

vectorize_layer = TextVectorization(output_mode='int', max_tokens=vocab_size, pad_to_max_tokens=True, output_sequence_length=256)
vectorize_layer.adapt(X_train)
clf = cnn_embedding_softmax_model(vectorize_layer)
print(clf.summary())
clf.compile(loss='categorical_crossentropy', metrics=['accuracy'])
history = clf.fit(X_train, y_train, epochs=5, verbose=1, validation_split=0.1)
clf.evaluate(X_test, y_test)

Model: "model_94"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_105 (InputLayer)      [(None, 1)]               0         
                                                                 
 text_vectorization_21 (Text  (None, 256)              0         
 Vectorization)                                                  
                                                                 
 projecao (Embedding)        (None, 256, 2)            2000      
                                                                 
 ngramas (Functional)        (None, 128, 16)           96        
                                                                 
 global_average_pooling1d_10  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 classificador (Dense)       (None, 2)                 34 

[0.3568091094493866, 0.8440799713134766]

## Exercício 3
**Objetivo: visualizar os filtros definidos na CNN**

O código abaixo mostra uma figura que sobrepõe os embeddings encontrados aos filtros definidos na CNN. Com base na figura, você diria que o sistema está contanto palavras, ou que está usando composições de palavras para fazer sua predição?

In [91]:
import plotly.express as px

# Visualização: onde foi parar cada palavra?
projecoes = clf.get_layer('projecao').get_weights()[0]
vocabulario = vectorize_layer.get_vocabulary()
y_pred_ohe = clf.predict(vocabulario)
y_pred = ohe.inverse_transform(y_pred_ohe)


df = pd.DataFrame()
df['dim_1'] = projecoes[:,0]
df['dim_2'] = projecoes[:,1]
df['word'] = vocabulario
df['prediction'] = y_pred.reshape( (-1,))


fig  = px.scatter(df, x="dim_1", y="dim_2", color="prediction", hover_data=["word"], title="Onde foi cada palavra?", width=600, height=600)
# E onde foi parar cada ngrama capturado?
ngramas = clf.get_layer('ngramas').get_weights()[0]
for i in range(ngramas.shape[2]):
    fig.add_scatter(x=ngramas[:,0,i], y=ngramas[:,1,i], mode='lines')
fig.show()


32/32 [==============================] - 0s 4ms/step


## Exercício 4
**Objetivo: analisar qualitativamente o resultado da rede convolucional**

Use o código abaixo para tentar escrever reviews que seriam classificados como positivos ou negativos. É possível fazer isso somente inserindo palavras específicas?

In [101]:
frases = ["dumb",
          "an action movie",
          "attempt",
          "dumb attempt to make an action movie",
          "funny",
          "trying to be funny",
          "bad",
          "a bad take on trying to be funny",
          "boring",
          "this is a boring movie"]

ohe.inverse_transform(clf.predict(frases))

1/1 [==============================] - 0s 79ms/step


array([['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['negative']], dtype=object)

## Exercício 5
**Objetivo: usar uma CNN profunda**

Uma vez que cada camada convolucional reduz nossa dimensão do tempo pela metade, é possível usarmos várias camadas desse tipo em sequência até encontrarmos como resultado um único passo de tempo, como no exemplo abaixo.

1. Nessa situação, como a curva de treinamento se comporta?
1. Modifique um dos parâmetros da rede (o número de ngramas ou o tamanho dos ngramas) e realize o treino novamente. Qual foi o impacto no desempenho final?

In [93]:
vocab_size = 1000
def deep_cnn_embedding_softmax_model(vectorize_layer, vocab_size=vocab_size, number_of_ngrams=16, n_gram_size=3):
    input_layer = Input(shape=(1,), dtype=tf.string)
    x = input_layer
    x = vectorize_layer(x)
    x = Embedding(vocab_size, 2, name='projecao')(x)
    x = convolve_and_downsample(256, 2, number_of_ngrams, n_gram_size, name='ngramas')(x)
    x = convolve_and_downsample(128, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(64, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(32, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(16, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(8, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(4, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = convolve_and_downsample(2, number_of_ngrams, number_of_ngrams, n_gram_size)(x)
    x = Reshape( (-1,))(x)
    x = Dense(2, name='classificador')(x)
    x = Activation('softmax')(x)
    return Model(input_layer, x)

vectorize_layer = TextVectorization(output_mode='int', max_tokens=vocab_size, pad_to_max_tokens=True, output_sequence_length=256)
vectorize_layer.adapt(X_train)
clf2 = deep_cnn_embedding_softmax_model(vectorize_layer)
print(clf2.summary())
clf2.compile(loss='categorical_crossentropy', metrics=['accuracy'])
history = clf2.fit(X_train, y_train, epochs=5, verbose=1, validation_split=0.1)
clf2.evaluate(X_test, y_test)

Model: "model_102"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_107 (InputLayer)      [(None, 1)]               0         
                                                                 
 text_vectorization_22 (Text  (None, 256)              0         
 Vectorization)                                                  
                                                                 
 projecao (Embedding)        (None, 256, 2)            2000      
                                                                 
 ngramas (Functional)        (None, 128, 16)           96        
                                                                 
 model_95 (Functional)       (None, 64, 16)            768       
                                                                 
 model_96 (Functional)       (None, 32, 16)            768       
                                                         

[0.3642828166484833, 0.8354399800300598]

## Exercício 6
**Objetivo: Visualizar n-gramas encontrados pela rede convolucional**

O código abaixo mostra os n-gramas gerados pela rede convolucional após o treinamento. Com base neles, você diria que a rede está contando palavras como um regressor logístico?

In [94]:
import plotly.express as px

# Visualização: onde foi parar cada palavra?
projecoes = clf2.get_layer('projecao').get_weights()[0]
vocabulario = vectorize_layer.get_vocabulary()
y_pred_ohe = clf2.predict(vocabulario)
y_pred = ohe.inverse_transform(y_pred_ohe)


df = pd.DataFrame()
df['dim_1'] = projecoes[:,0]
df['dim_2'] = projecoes[:,1]
df['word'] = vocabulario
df['prediction'] = y_pred.reshape( (-1,))


fig  = px.scatter(df, x="dim_1", y="dim_2", color="prediction", hover_data=["word"], title="Onde foi cada palavra?", width=600, height=600)
# E onde foi parar cada ngrama capturado?
ngramas = clf2.get_layer('ngramas').get_weights()[0]
for i in range(ngramas.shape[2]):
    fig.add_scatter(x=ngramas[:,0,i], y=ngramas[:,1,i], mode='lines')

fig.show()


32/32 [==============================] - 1s 7ms/step


## Exercício 7
**Objetivo: compor reviews positivos e negativos de propósito**

À partir dos resultados da célula a seguir, e dos plots anteriores, o que você diria que é necessário para fazer um review que seria avaliado como negativo ou positivo pelo nosso classificador? Após, tente compor novos reviews positivos e negativos!

In [99]:
frases = ["dumb",
          "an action movie",
          "attempt",
          "dumb attempt to make an action movie",
          "funny",
          "trying to be funny",
          "poor",
          "a poor take on trying to be funny",
          "boring",
          "this is a boring movie"]

ohe.inverse_transform(clf2.predict(frases))

1/1 [==============================] - 0s 89ms/step


array([['positive'],
       ['positive'],
       ['positive'],
       ['negative'],
       ['positive'],
       ['positive'],
       ['positive'],
       ['negative'],
       ['positive'],
       ['negative']], dtype=object)

# Exercício 8
**Objetivo: analisar a importância da ordem das palavras nos reviews**

O código abaixo avalia nosso classificador usando reviews com palavras embaralhadas.

1. De acordo com os resultados, a ordem das palavras é importante para encontrar a classificação do documento?
1. Como isso se compara com os resultados do exercício anterior?
1. Justifique esses resultados tomando por base o funcionamento do classificador.

In [102]:
import random
X_test_shuffle = pd.Series([' '.join( random.sample(X_test.iloc[i].split(), len(X_test.iloc[i].split())) ) for i in range(len(X_test))])
clf2.evaluate(X_test_shuffle, y_test)

391/391 [==============================] - 9s 22ms/step - loss: 0.3560 - accuracy: 0.8418


[0.356004536151886, 0.841759979724884]

## Exercício 9
**Objetivo: comparar os classificadores que já estudamos até o momento**

Até o momento, sabemos trabalhar com os classificadores Naive Bayes, Logistic Regression, redes neurais com embedding e global average pooling, redes neurais recorrentes (3 tipos: RNN, GRU e LSTM) e redes convolucionais profundas. 

Ordene os métodos de classificação em termos de:

1. Acurácia
1. Recursos computacionais necessários
1. Explicabilidade, isto é, quão difícil é explicar os resultados da rede


## Exercício 10
**Objetivo: praticar o uso de camadas de rede para compor redes neurais**

Implemente e teste uma rede neural que funciona da seguinte forma:

1. As entradas são textos que são vetorizados e as palavras são colocadas em embeddings de 2 dimensões
1. Há uma camada de convolução e sub-amostragem que calcula tetragramas do texto e o subamostra no tempo de um fator de 4.
1. O resultado dessa sub-amostragem é resumido no tempo usando uma LSTM bidirecional.
1. Por fim, o resumo é classificado usando uma rede fully-connected com ativação softmax.

Faça o treinamento da sua rede para uma base de dados à sua escolha (pode ser a IMDB). Analisando as curvas de aprendizado, defina quantas épocas são necessárias para o treinamento, e verifique se você detecta algum tipo de overfitting no processo. Por fim, mostre o desempenho da rede usando a acurácia num conjunto de teste.